In [2]:
import gymnasium as gym
env = gym.make("Walker2d-v5")
env.reset()
print("Mujoco environment initialized successfully!")

Mujoco environment initialized successfully!


In [3]:
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# 创建 Walker2d 环境
env = gym.make("Walker2d-v5")

# 初始化 PPO 代理
model = PPO("MlpPolicy", env, verbose=1)

# 训练 100k 步
model.learn(total_timesteps=100000)

# 评估代理性能
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward}, Std: {std_reward}")

# 保存模型
model.save("walker2d_ppo")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


RuntimeError: Numpy is not available

In [ ]:
import time
import gymnasium as gym
from stable_baselines3 import PPO

# 创建可视化环境
env = gym.make("Walker2d-v5", render_mode="human")  # 改成 v5 以保持最新
model = PPO.load("walker2d_ppo")

obs, _ = env.reset()
done = False
while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, _ = env.step(action)
    env.render()
    time.sleep(0.01)

env.close()

In [ ]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO

# 加载训练好的模型
model = PPO.load("walker2d_ppo")

# 创建 Walker2d 环境
env = gym.make("Walker2d-v5")

# 评估 Walker2d 代理
n_episodes = 10  # 评估 10 轮
episode_rewards = []
episode_lengths = []

for i in range(n_episodes):
    obs, _ = env.reset()
    done = False
    total_reward = 0
    steps = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _, _ = env.step(action)
        total_reward += reward
        steps += 1

    episode_rewards.append(total_reward)
    episode_lengths.append(steps)

# 计算评估结果
mean_reward = np.mean(episode_rewards)
std_reward = np.std(episode_rewards)
mean_length = np.mean(episode_lengths)

print(f"🏆 Walker2d 训练评估结果（基于 {n_episodes} 轮测试）：")
print(f"✅ 平均奖励（Mean Reward）: {mean_reward:.2f} ± {std_reward:.2f}")
print(f"✅ 平均步数（Mean Episode Length）: {mean_length:.2f} steps")
print(f"✅ 单轮最大奖励（Max Episode Reward）: {np.max(episode_rewards):.2f}")
print(f"✅ 单轮最小奖励（Min Episode Reward）: {np.min(episode_rewards):.2f}")

# 判断 Walker 是否训练成功
if mean_reward > 200:
    print("🎉 Walker2d 训练效果良好！Walker 能较稳定地行走。")
elif mean_reward > 100:
    print("⚠️ Walker2d 训练一般，可能偶尔摔倒或动作不自然，可以考虑增加训练步数。")
else:
    print("❌ Walker2d 训练较差，Walker 可能不能行走，建议调整超参数或训练更久。")


ModuleNotFoundError: No module named 'numpy._core.numeric'

In [ ]:
import gymnasium as gym
import numpy as np
import cv2
from stable_baselines3 import PPO

# 加载训练好的 PPO 模型
model = PPO.load("walker2d_ppo")

# 创建环境（启用录像）
env = gym.make("Walker2d-v5", render_mode="rgb_array")

# 录制视频
video_frames = []
n_episodes = 5  # 录制 5 轮

episode_rewards = []
episode_lengths = []

for episode in range(n_episodes):
    obs, _ = env.reset()
    done = False
    total_reward = 0
    steps = 0

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _, _ = env.step(action)
        total_reward += reward
        steps += 1

        # 记录视频帧
        frame = env.render()
        video_frames.append(frame)

    episode_rewards.append(total_reward)
    episode_lengths.append(steps)

# 释放环境
env.close()

# 保存视频
video_path = "walker2d_run.mp4"

# 将帧数据写入 MP4 文件
height, width, _ = video_frames[0].shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_path, fourcc, 30, (width, height))

for frame in video_frames:
    out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

out.release()

# 计算 Walker2d 运行的统计数据
mean_reward = np.mean(episode_rewards)
std_reward = np.std(episode_rewards)
mean_length = np.mean(episode_lengths)

print(f"🏆 Walker2d 运行视频已录制，视频文件：{video_path}")
print(f"✅ 平均奖励（Mean Reward）: {mean_reward:.2f} ± {std_reward:.2f}")
print(f"✅ 平均步数（Mean Episode Length）: {mean_length:.2f} steps")
print(f"✅ 单轮最大奖励（Max Episode Reward）: {np.max(episode_rewards):.2f}")
print(f"✅ 单轮最小奖励（Min Episode Reward）: {np.min(episode_rewards):.2f}")
